## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = range(1,1001)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [5]:
rdd = sc.parallelize(data,numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [6]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [7]:
# count
rdd.count()

1000

In [8]:
# first
rdd.first()

1

In [10]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [11]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [12]:
## Note: When you are dealing with big data, this could make your computer crash! It's best to avoid using the collect() method
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [15]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([4.91162090e-01, 2.82202915e-01, 8.38864963e-01, 1.70897522e+00,
       7.25437710e-01, 5.55545278e+00, 5.11605189e+00, 1.84576288e+00,
       7.72849361e+00, 2.76244579e+00, 5.71143184e+00, 9.08816755e+00,
       6.07298242e+00, 7.99307340e+00, 5.75971121e+00, 7.69835290e+00,
       7.93114673e+00, 1.58482105e+01, 6.24555167e+00, 7.35224142e+00,
       2.53617964e+00, 2.70001892e-01, 1.83573024e+01, 1.12062900e+01,
       1.33782330e+01, 7.96200746e+00, 3.98211311e+00, 7.56692715e+00,
       5.10397236e+00, 1.27540671e+01, 2.97610186e+01, 1.49186555e+01,
       1.36619828e+01, 3.54694224e+00, 1.50258842e+01, 1.46490878e+01,
       1.82018672e+00, 1.78250695e+01, 3.87675240e+01, 2.88149722e+01,
       1.12290849e+01, 1.69512104e+01, 2.61430776e+01, 2.97980425e+01,
       2.20237389e+01, 6.10909509e+00, 1.40789942e+01, 2.62971865e+01,
       4.81111597e+01, 1.10943450e+01, 5.01403210e+00, 2.59181440e+01,
       2.70918083e+01, 8.12104304e+00, 3.43447641e+01, 5.69170623e-01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [16]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.49116209035369984,
 0.2822029149691465,
 0.8388649633854133,
 1.7089752202019763]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [17]:
def sales_tax(num):
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [18]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.4518691231254039,
 0.2596266817716148,
 0.7717557663145802,
 1.5722572025858184,
 0.6674026933609911,
 5.111016561693545,
 4.706767741130603,
 1.6981018492599291,
 7.1102141177005755,
 2.5414501302888195]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [19]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [20]:
discounted.take(10)

[0.4066822108128635,
 0.23366401359445332,
 0.6945801896831222,
 1.4150314823272365,
 0.600662424024892,
 4.599914905524191,
 4.236090967017543,
 1.5282916643339362,
 6.399192705930518,
 2.2873051172599377]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [21]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[807.5223868620121,
 803.693874551964,
 797.0959074152563,
 772.4720933167146,
 772.3331001250755,
 768.2195457481905,
 756.2294835247766,
 748.8476486384122,
 744.0619471237792,
 739.4728027032282,
 733.6720434928918,
 732.9012742965945,
 731.4481729387568,
 721.5721263993756,
 715.3312084160882]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [22]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[6] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [23]:
mapped =price_items.map(lambda x : (x,x*0.92*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.49116209035369984, 0.4066822108128635), (0.2822029149691465, 0.23366401359445332), (0.8388649633854133, 0.6945801896831222), (1.7089752202019763, 1.4150314823272365), (0.7254377101749904, 0.600662424024892), (5.555452784449505, 4.599914905524191), (5.116051892533264, 4.236090967017543), (1.8457628796303576, 1.5282916643339362), (7.7284936061962775, 6.399192705930518), (2.762445793792195, 2.2873051172599377)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [24]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.49116209035369984, 0.4066822108128635, 0.2822029149691465, 0.23366401359445332, 0.8388649633854133, 0.6945801896831222, 1.7089752202019763, 1.4150314823272365, 0.7254377101749904, 0.600662424024892]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [27]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

252

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [28]:
selected_items.reduce(lambda x,y :x+y)

116727.08046894378

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [29]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(17, 300.8325066039031),
 (29, 305.25766441038667),
 (17, 305.0977844132455),
 (21, 304.98392128039535),
 (5, 333.8857682319069),
 (14, 305.5898626401891),
 (13, 308.11937859505696)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [30]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y:x+y)
total_spent.take(10)

[(10, 2581.92030696869),
 (40, 2593.439202467493),
 (20, 4049.5862837755344),
 (30, 1339.187953168383),
 (50, 3276.085779221423),
 (21, 2243.510614990998),
 (11, 961.7459934621598),
 (41, 2613.0660906051685),
 (1, 2227.250514154455),
 (31, 1954.8474710250343)]

In [31]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending = False).collect()

[(26, 5798.579818731461),
 (4, 4527.52693665395),
 (20, 4049.5862837755344),
 (38, 3819.7597555728435),
 (46, 3802.8498853270303),
 (34, 3677.1926107308227),
 (43, 3653.14057239865),
 (32, 3561.9281943436563),
 (33, 3453.6142025283175),
 (50, 3276.085779221423),
 (8, 3258.1378289736986),
 (6, 3238.622324459873),
 (7, 2880.179218032851),
 (29, 2737.227881450081),
 (12, 2629.7775261042234),
 (41, 2613.0660906051685),
 (40, 2593.439202467493),
 (10, 2581.92030696869),
 (15, 2541.419452255079),
 (28, 2386.920659881679),
 (27, 2385.829211830839),
 (16, 2337.3419736943542),
 (22, 2302.708278369993),
 (5, 2276.921434953737),
 (21, 2243.510614990998),
 (1, 2227.250514154455),
 (45, 2153.887271748221),
 (48, 2080.685937371077),
 (31, 1954.8474710250343),
 (25, 1900.2202373100217),
 (3, 1898.5737028764197),
 (13, 1807.2565949253087),
 (37, 1785.120061282712),
 (49, 1739.9576916006727),
 (9, 1681.233644596207),
 (19, 1639.620961269848),
 (35, 1622.6821826378182),
 (24, 1579.2481548565065),
 (36, 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [33]:
total_items = sales_data.countByKey()
total_items.items()

dict_items([(14, 7), (24, 7), (17, 8), (31, 5), (10, 6), (44, 7), (33, 5), (25, 5), (29, 6), (28, 5), (1, 9), (36, 7), (6, 2), (41, 4), (47, 8), (40, 6), (34, 3), (32, 8), (15, 6), (21, 3), (4, 4), (2, 5), (5, 5), (18, 3), (27, 7), (45, 8), (46, 5), (3, 6), (37, 6), (8, 6), (9, 3), (22, 4), (38, 5), (48, 9), (12, 5), (49, 6), (20, 2), (30, 4), (16, 5), (13, 3), (19, 3), (7, 5), (26, 4), (11, 5), (42, 1), (50, 5), (23, 3), (39, 3), (43, 2), (35, 3)])

In [35]:
total_items

defaultdict(int,
            {11: 5,
             39: 6,
             40: 5,
             47: 6,
             29: 11,
             26: 3,
             17: 6,
             13: 7,
             3: 5,
             46: 3,
             20: 5,
             31: 6,
             42: 4,
             19: 5,
             36: 7,
             25: 8,
             44: 7,
             12: 7,
             1: 3,
             16: 5,
             38: 4,
             4: 6,
             30: 5,
             50: 10,
             49: 5,
             22: 7,
             37: 9,
             10: 6,
             45: 2,
             5: 6,
             27: 8,
             41: 5,
             35: 7,
             15: 3,
             23: 1,
             32: 2,
             14: 4,
             8: 4,
             24: 3,
             21: 6,
             33: 3,
             2: 4,
             6: 5,
             9: 4,
             43: 5,
             48: 4,
             28: 5,
             7: 2,
             34: 2,
          

In [34]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda kv:kv[1], reverse =True)

[(29, 11),
 (50, 10),
 (37, 9),
 (25, 8),
 (27, 8),
 (13, 7),
 (36, 7),
 (44, 7),
 (12, 7),
 (22, 7),
 (35, 7),
 (39, 6),
 (47, 6),
 (17, 6),
 (31, 6),
 (4, 6),
 (10, 6),
 (5, 6),
 (21, 6),
 (11, 5),
 (40, 5),
 (3, 5),
 (20, 5),
 (19, 5),
 (16, 5),
 (30, 5),
 (49, 5),
 (41, 5),
 (6, 5),
 (43, 5),
 (28, 5),
 (42, 4),
 (38, 4),
 (14, 4),
 (8, 4),
 (2, 4),
 (9, 4),
 (48, 4),
 (26, 3),
 (46, 3),
 (1, 3),
 (15, 3),
 (24, 3),
 (33, 3),
 (45, 2),
 (32, 2),
 (7, 2),
 (34, 2),
 (23, 1),
 (18, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
